In [1]:
import requests
import json
from transformers import pipeline

1. Collecting Lyrics Using Musixmatch API

In [2]:
import requests
import json

api_key = 'b76a1b9a47064853d3363d46e5352e79'

# Function to get lyrics from Musixmatch
def get_lyrics(song_name, artist_name, api_key):
    base_url = "https://api.musixmatch.com/ws/1.1/"
    
    #Endpoint to search for the track
    search_url = f"{base_url}track.search"
    search_params = {
        'q_track': song_name,
        'q_artist': artist_name,
        'apikey': api_key,
        'f_has_lyrics': 1
    }
    
    # Making the request to search for the track
    response = requests.get(search_url, params=search_params)
    data = response.json()
    
    if data['message']['header']['status_code'] != 200:
        raise Exception("API request failed")
    
    track_list = data['message']['body']['track_list']
    if not track_list:
        return None
    
    # Getting the track ID of the first result
    track_id = track_list[0]['track']['track_id']
    
    # Endpoint to get lyrics
    lyrics_url = f"{base_url}track.lyrics.get"
    lyrics_params = {
        'track_id': track_id,
        'apikey': api_key
    }
    
    # Making the request to get the lyrics
    response = requests.get(lyrics_url, params=lyrics_params)
    data = response.json()
    
    if data['message']['header']['status_code'] != 200:
        raise Exception("API request failed")
    
    lyrics = data['message']['body']['lyrics']['lyrics_body']
    return lyrics

# Example usage
api_key = 'b76a1b9a47064853d3363d46e5352e79'
song_name = "Shape of You"
artist_name = "Ed Sheeran"
lyrics = get_lyrics(song_name, artist_name, api_key)
print(lyrics)

The club isn't the best place to find a lover
So the bar is where I go (mm-mm)
Me and my friends at the table doing shots
Drinking fast, and then we talk slow (mm-mm)
You come over and start up a conversation with just me
And trust me, I'll give it a chance now (mm-mm)
Take my hand, stop, put "Van the Man" on the jukebox
And then we start to dance
And now I'm singing like

Girl, you know I want your love
Your love was handmade for somebody like me
Come on now, follow my lead
I may be crazy, don't mind me
Say, "Boy, let's not talk too much
Grab on my waist and put that body on me"
Come on now, follow my lead
Come, come on now, follow my lead
(Mm-mm)

I'm in love with the shape of you
We push and pull like a magnet do
Although my heart is falling too
I'm in love with your body
Last night you were in my room
And now my bed sheets smell like you
Every day discovering something brand new
Oh, I'm in love with your body

Oh I, oh I, oh I, oh I
Oh, I'm in love with your body
Oh I, oh I, oh I, 

2. Sentiment Analysis on Lyrics Using NLP (e.g., BERT)

In [ ]:
from transformers import pipeline

# Load pre-trained sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to get sentiment of lyrics
def analyze_sentiment(lyrics):
    sentiments = sentiment_pipeline(lyrics)
    return sentiments

# Example usage
lyrics = "Your song lyrics here..."
sentiment = analyze_sentiment(lyrics)
print(sentiment)


3. Fetching Similar Songs on Spotify Based on Sentiment

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify API credentials
spotify_client_id = 'YOUR_SPOTIFY_CLIENT_ID'
spotify_client_secret = 'YOUR_SPOTIFY_CLIENT_SECRET'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_client_id,
                                                           client_secret=spotify_client_secret))

# Function to get song recommendations based on seed track
def get_song_recommendations(seed_tracks, target_valence, limit=10):
    recommendations = sp.recommendations(seed_tracks=seed_tracks, target_valence=target_valence, limit=limit)
    return recommendations['tracks']

# Example usage
seed_tracks = ['spotify_track_id_of_the_seed_song']  # You need the track ID from Spotify
target_valence = 0.5  # A value between 0 (sad) and 1 (happy)
recommendations = get_song_recommendations(seed_tracks, target_valence)
for track in recommendations:
    print(f"Track: {track['name']} by {track['artists'][0]['name']}")


Integration
Create a main script to tie everything together.

In [ ]:
import requests
from transformers import pipeline
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Define API keys and client secrets
musixmatch_api_key = 'YOUR_MUSIXMATCH_API_KEY'
spotify_client_id = 'YOUR_SPOTIFY_CLIENT_ID'
spotify_client_secret = 'YOUR_SPOTIFY_CLIENT_SECRET'

# Initialize Spotify API client
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_client_id,
                                                           client_secret=spotify_client_secret))

# Initialize sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to get lyrics from Musixmatch
def get_lyrics(song_name, artist_name, api_key):
    base_url = "https://api.musixmatch.com/ws/1.1/"
    search_url = f"{base_url}track.search?q_track={song_name}&q_artist={artist_name}&apikey={api_key}&f_has_lyrics=1"
    
    response = requests.get(search_url)
    data = response.json()
    
    if data['message']['header']['status_code'] != 200:
        raise Exception("API request failed")
    
    track_list = data['message']['body']['track_list']
    if not track_list:
        return None
    
    track_id = track_list[0]['track']['track_id']
    lyrics_url = f"{base_url}track.lyrics.get?track_id={track_id}&apikey={api_key}"
    response = requests.get(lyrics_url)
    data = response.json()
    
    if data['message']['header']['status_code'] != 200:
        raise Exception("API request failed")
    
    lyrics = data['message']['body']['lyrics']['lyrics_body']
    return lyrics

# Function to analyze sentiment of lyrics
def analyze_sentiment(lyrics):
    sentiments = sentiment_pipeline(lyrics)
    return sentiments

# Function to get song recommendations based on seed track
def get_song_recommendations(seed_tracks, target_valence, limit=10):
    recommendations = sp.recommendations(seed_tracks=seed_tracks, target_valence=target_valence, limit=limit)
    return recommendations['tracks']

# Main function to get recommendations based on user input
def main():
    song_name = input("Enter the song name: ")
    artist_name = input("Enter the artist name: ")
    
    lyrics = get_lyrics(song_name, artist_name, musixmatch_api_key)
    if lyrics:
        sentiment = analyze_sentiment(lyrics)
        target_valence = 0.5  # This can be adjusted based on sentiment analysis results
        seed_tracks = ['spotify_track_id_of_the_seed_song']  # You need the track ID from Spotify
        
        recommendations = get_song_recommendations(seed_tracks, target_valence)
        print("Recommended songs based on sentiment:")
        for track in recommendations:
            print(f"Track: {track['name']} by {track['artists'][0]['name']}")
    else:
        print("Lyrics not found for the given song.")

if __name__ == "__main__":
    main()


Notes
Replace placeholders (e.g., YOUR_MUSIXMATCH_API_KEY, YOUR_SPOTIFY_CLIENT_ID, YOUR_SPOTIFY_CLIENT_SECRET) with actual API keys.
For the Spotify recommendations, you need the Spotify track ID of the seed song. You can get it by searching for the song on Spotify and extracting the track ID from the URL.
Adjust the target_valence based on the sentiment analysis results to refine recommendations.
Enhance the main function to handle different sentiment scores and provide dynamic target_valence values.
By following these steps and using the provided code, you'll be able to create a comprehensive project that analyzes song lyrics' sentiment and recommends similar songs based on that sentiment.